## Model Training

In [10]:
import pandas as pd
import numpy as np
# Handling missing values and feature scaling
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
df = pd.read_csv('data/2023_04-data-maresme.csv')
df.head()

,id,time,province,county,city,area,neighborhood,title,type,price,parking,rooms,sqrm,floor,surface,elevator,tag,img,url
0,99006221,2023-04-01,Barcelona,Maresme,Arenys de Munt,NaN,NaN,Chalet en Arenys de Munt,venta,360000,1,3.0,182,NaN,NaN,NaN,NaN,https://img3.idealista.com/blur/WEB_LISTING-M/...,https://www.idealista.com/inmueble/99006221/
1,100404805,2023-04-01,Barcelona,Maresme,Arenys de Munt,NaN,NaN,Casa o chalet independiente en Arenys de Munt,venta,570000,0,5.0,266,NaN,NaN,NaN,NaN,https://img3.idealista.com/blur/WEB_LISTING-M/...,https://www.idealista.com/inmueble/100404805/
2,84948994,2023-04-01,Barcelona,Maresme,Arenys de Munt,NaN,NaN,"Chalet en riera i Penya, Arenys de Munt",venta,220000,0,8.0,320,NaN,NaN,NaN,NaN,https://img3.idealista.com/blur/WEB_LISTING-M/...,https://www.idealista.com/inmueble/84948994/
3,92395817,2023-04-01,Barcelona,Maresme,Arenys de Munt,NaN,NaN,Casa o chalet independiente en Urbanització Co...,venta,495000,1,6.0,199,NaN,NaN,NaN,NaN,https://img3.idealista.com/blur/WEB_LISTING-M/...,https://www.idealista.com/inmueble/92395817/
4,95768293,2023-04-01,Barcelona,Maresme,Arenys de Munt,NaN,NaN,Chalet en Arenys de Munt,venta,575000,1,5.0,202,NaN,NaN,NaN,Lujo,https://img3.idealista.com/blur/WEB_LISTING-M/...,https://www.idealista.com/inmueble/95768293/


In [12]:
# Copy paste the data cleaning code from the EDA notebook
df['rooms'].fillna(value=df['rooms'].mean(), inplace=True)
df['rooms'] = df['rooms'].astype(int)
df['sqrm'] = df['sqrm'].replace('[^0-9]', '', regex=True)
df['sqrm'] = df['sqrm'].replace('', np.nan)
df['sqrm'] = df['sqrm'].astype(float)
df['sqrm'].fillna(value=df['sqrm'].mean())
df['sqrm'] = df['sqrm'].replace(np.nan, df['sqrm'].mean())
df['sqrm'] = df['sqrm'].astype(int)
df['floor'] = df['floor'].replace('-', '-1')
df['floor']=df['floor'].fillna('0')
df['floor'] = df['floor'].astype(int)
df['elevator'] = df['elevator'].fillna(0)
df['elevator'] = df['elevator'].astype(int)
df['property_type'] = [x.split()[0] for x in df['title']]
df['property_type'].value_counts()
df['province'] = df['province'].fillna('empty')
df['county'] = df['county'].fillna('empty')
df['city'] = df['city'].fillna('empty')
df['area'] = df['area'].fillna('empty')
df['neighborhood'] = df['neighborhood'].fillna('empty')
df['location'] = df['province'] + '-' + df['county'] + '-' + df['city'] + '-' + df['area'] + '-' + df['neighborhood']
df=df.drop(labels=['id', 'time', 'province', 'county', 'area', 'city', 'neighborhood', 'title', 'img', 'url', 'surface', 'tag'], axis=1)
df.rename(columns={'type': 'deal_type'}, inplace=True)
df=df.drop_duplicates()

In [13]:
# df for rent and another one for sale
df_venta = df[df['deal_type']=='venta']
df_alquiler = df[df['deal_type']=='alquiler']

In [14]:
# independent and dependent features for rent
X_alquiler = df_alquiler.drop(labels=['price'], axis=1)
Y_alquiler = df_alquiler[['price']]

In [15]:
# Define which columns should be ordinal-encoded and which shuould be scaled
categorical_cols = X_alquiler.select_dtypes(include='object').columns
numerical_cols = X_alquiler.select_dtypes(exclude='object').columns

In [16]:
numerical_cols

Index(['parking', 'rooms', 'sqrm', 'floor', 'elevator'], dtype='object')

In [17]:
categorical_cols

Index(['deal_type', 'property_type', 'location'], dtype='object')

In [17]:
# Handling missing values and feature scaling
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
## Numerical pipline that handles missing values and scaling
num_pipeline = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
    ]
)

# Categorical Pipeline

cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehotencoder', OneHotEncoder())
    ]
)

In [ ]:
# Eliminar fincas